In [1]:
import pickle as pkl
import numpy as np
import tensorflow as tf
import torch
import os
from PIL import Image
import torchvision
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
import torchvision.transforms as transforms
import torch.nn as nn
from torch.backends import cudnn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F

from classifiers.joint_classifier import MnistClassifier
from dataloaders.concat_dataset import ConcatDataset
from dataloaders.mnistm_loader import MNIST_M
from utils.train_utils import train_mnist_joint

# global is_gpu
# is_gpu = torch.cuda.is_available()

root_dir = "../"
data_dir = os.path.join(root_dir,"datasets/")
mnistm_dir = os.path.join(root_dir,"datasets/mnistm/")

target_train_batch_size = 32
target_test_batch_size = 512
source_train_batch_size = 32
source_test_batch_size = 512

target_transform = transforms.Compose([transforms.ToTensor()])
source_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.1307,), (0.3081,))])

target_train_dataset = MNIST_M(root=mnistm_dir, train=True, transform=target_transform)
target_test_dataset = MNIST_M(root=mnistm_dir, train=False, transform=target_transform)

print('Size of target train dataset: %d' % len(target_train_dataset))
print('Size of target test dataset: %d' % len(target_test_dataset))


source_train_set = torchvision.datasets.MNIST(root=data_dir, train=True,
                                        download=True, transform=source_transform)
source_test_set = torchvision.datasets.MNIST(root=data_dir, train=False,
                                        download=True, transform=source_transform)

print('Size of source train dataset: %d' % len(target_train_dataset))
print('Size of source test dataset: %d' % len(target_test_dataset))

train_loader = torch.utils.data.DataLoader(
            ConcatDataset(source_train_set,target_train_dataset),
            batch_size=source_train_batch_size, shuffle=False, pin_memory=True)
test_loader = torch.utils.data.DataLoader(
            ConcatDataset(source_test_set,target_test_dataset),
            batch_size=source_test_batch_size, shuffle=True, pin_memory=True)
gpu_list = [1,2]
net = MnistClassifier(1, 3, 10,gpu_list,2)
# if is_gpu:
net.cuda(gpu_list[0])

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

train_mnist_joint(net,train_loader,train_loader,optimizer,criterion,epochs = 5,early_stop = 3,schedule = [0.5,])

Size of target train dataset: 55000
Size of target test dataset: 10000
Size of source train dataset: 55000
Size of source test dataset: 10000
Starting to train the classifier on both the datasets
[1/5] Statistics - per epoch time: 18.67,classification loss: 4.601
Test Statistics - time: 0.75
Accuracy of the network on the source test images: 13.54 %
Accuracy of the network on the target test images: 11.46 %
[2/5] Statistics - per epoch time: 4.44,classification loss: 4.600
Test Statistics - time: 0.75
Accuracy of the network on the source test images: 13.54 %
Accuracy of the network on the target test images: 11.46 %
Scheduling the rate from 0.001000 to 0.000100
[3/5] Statistics - per epoch time: 0.71,classification loss: 4.599
Test Statistics - time: 0.67
Accuracy of the network on the source test images: 13.54 %
Accuracy of the network on the target test images: 11.46 %
[4/5] Statistics - per epoch time: 0.76,classification loss: 4.598
Test Statistics - time: 2.26
Accuracy of the net